In [20]:
cd

C:\Users\torye


c:\Users\torye\anaconda3\envs\GPU\lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
c:\Users\torye\anaconda3\envs\GPU\lib\site-packages\IPython\core\magics\osm.py:428: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [21]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import shutil
import csv

from tensorflow.keras import layers, models, Input
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, Callback

#tf.config.set_visible_devices([], 'GPU')

# Attack Dataset Load

In [22]:
#df_gear = pd.read_csv('./work/HyDL_dataset/CAN/gear_dataset_HyDL.csv')
#df_rpm = pd.read_csv('./work/HyDL_dataset/CAN/RPM_dataset_HyDL.csv')
#df_dos = pd.read_csv('./work/HyDL_dataset/CAN/DoS_dataset_HyDL.csv')
#df_fuzzy = pd.read_csv('./work/HyDL_dataset/CAN/Fuzzy_dataset_HyDL.csv')

#df = pd.read_csv('./work/Survival_and_Challenge_2019/HYUNDAI_Sonata/HY_Sonata_Replay_DEC.csv')
df = pd.read_csv('./work/Survival_and_Challenge_2019/KIA_Soul/KIA_Soul_Replay_DEC.csv')



#df = df.iloc[:100]

protocol = "CAN"
#protocol = "CAN-FD"

#attack = "gear"
#attack = "RPM"
#attack = "DoS"
#attack = "Fuzzy"
attack = "Replay"

# Make folder

In [ ]:
#version = "HY_SONATA"
version = "KIA_SOUL"
#version = "HY_G80"

folder_name = f"HyDL_{protocol}_{version}"

directory = "./Models/22.HyDL_(VC)" 
path = os.path.join(directory, folder_name)



# Check that the folder already exists and Delete folder if it exists
if os.path.exists(path):
    shutil.rmtree(path) 

# Create a new folder
os.makedirs(path)

# Preprocessing

In [24]:
#======================================# 
print("Start")
#======================================# 

if protocol == "CAN-FD":
    num_data = 64
else:
    num_data = 8

df = df.drop(['Timestamp'], axis=1)
arbitration_id_array = df['Arbitration_ID'].to_numpy()

dlc_min_val = df['DLC'].min()
dlc_max_val = df['DLC'].max()

def normalize(val, min_val, max_val):
    if max_val == min_val:
        return 0
    else:
        return (np.log(val + 1) - np.log(min_val + 1)) / (np.log(max_val + 1) - np.log(min_val + 1))

df['DLC'] = df['DLC'].apply(lambda x: normalize(x, dlc_min_val, dlc_max_val))

for i in range(num_data):
    column = f'Data{i}'
    min_val = df[column].min()
    max_val = df[column].max()
    df[column] = df[column].apply(lambda x: normalize(x, min_val, max_val))


dlc_array = df['DLC'].to_numpy()
data_arrays = [df[f'Data{i}'].to_numpy() for i in range(num_data)]

arrays_to_concatenate = [arbitration_id_array] + [dlc_array] + data_arrays
concatenated_array = np.column_stack(arrays_to_concatenate)

if protocol == "CAN-FD":
    training = concatenated_array.reshape(-1, 66, 1)
else:
    training = concatenated_array.reshape(-1, 10, 1)

print(training.shape)

labeling = df['Class'].to_numpy()

#======================================# 
print("Done")
#======================================# 

Start
(566338, 10, 1)
Done


# Dataset Split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    training, labeling, test_size=0.33, random_state=42)


#======================================#  
print("Dataset Split : Done")
#======================================#  

Dataset Split : Done


# HyDL-IDS

In [26]:
def HyDL_IDS(input_shape, num_classes=1):
    inputs = Input(shape=input_shape)

    x = layers.Conv1D(32, 3, activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv1D(64, 3, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Dropout(0.2)(x)

    x = layers.LSTM(128, activation='tanh')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    outputs = layers.Dense(num_classes, activation='sigmoid')(x) 

    model = Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (66, 1) if protocol == "CAN-FD" else (10, 1)

model = HyDL_IDS(input_shape)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 10, 1)]           0         
                                                                 
 conv1d_4 (Conv1D)           (None, 8, 32)             128       
                                                                 
 batch_normalization_4 (Batc  (None, 8, 32)            128       
 hNormalization)                                                 
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 4, 32)            0         
 1D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 4, 32)             0         
                                                                 
 conv1d_5 (Conv1D)           (None, 2, 64)             6208

In [27]:
# Define a custom callback to log batch losses
class BatchLossCSVLogger(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs=None):
        self.losses.append(logs.get('loss'))


# Create an instance of the custom callback
batch_loss_logger = BatchLossCSVLogger()

# Define an EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1, mode='max', restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=10, batch_size=64,
                    validation_data=(X_test, y_test),
                    verbose=2,
                    callbacks=[batch_loss_logger, early_stopping])

Epoch 1/10
5929/5929 - 53s - loss: 0.5453 - accuracy: 0.7476 - val_loss: 0.4742 - val_accuracy: 0.7802 - 53s/epoch - 9ms/step
Epoch 2/10
5929/5929 - 49s - loss: 0.4844 - accuracy: 0.7771 - val_loss: 0.4559 - val_accuracy: 0.7921 - 49s/epoch - 8ms/step
Epoch 3/10
5929/5929 - 49s - loss: 0.4640 - accuracy: 0.7874 - val_loss: 0.4529 - val_accuracy: 0.8010 - 49s/epoch - 8ms/step
Epoch 4/10
5929/5929 - 49s - loss: 0.4420 - accuracy: 0.7996 - val_loss: 0.3975 - val_accuracy: 0.8205 - 49s/epoch - 8ms/step
Epoch 5/10
5929/5929 - 49s - loss: 0.4370 - accuracy: 0.8019 - val_loss: 0.4311 - val_accuracy: 0.7968 - 49s/epoch - 8ms/step
Epoch 6/10
5929/5929 - 49s - loss: 0.4236 - accuracy: 0.8069 - val_loss: 0.3846 - val_accuracy: 0.8160 - 49s/epoch - 8ms/step
Epoch 7/10
Restoring model weights from the end of the best epoch: 4.
5929/5929 - 49s - loss: 0.4233 - accuracy: 0.8065 - val_loss: 0.4674 - val_accuracy: 0.8025 - 49s/epoch - 8ms/step
Epoch 7: early stopping


In [28]:
# Write batch losses to a CSV file with a dynamic filename
file_path = os.path.join(path, f'{attack}_batch_losses.csv')
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Batch', 'Loss'])
    for i, loss in enumerate(batch_loss_logger.losses):
        writer.writerow([i, loss])
        
print(f"Batch validation losses saved to {file_path}")

Batch validation losses saved to ./Models/22.HyDL_(VC)\HyDL_CAN_HY_SONATA\Replay_batch_losses.csv


# Model Test

In [29]:
def evaluate_model(model, X_test, y_test, category, results):
    predicted_probabilities = model.predict(X_test)
    predicted_classes = (predicted_probabilities > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, predicted_classes)
    precision = precision_score(y_test, predicted_classes)
    recall = recall_score(y_test, predicted_classes)
    f1 = f1_score(y_test, predicted_classes)

    temp_df = pd.DataFrame(
        {'Category': [category], 'Accuracy': [accuracy], 'Precision': [precision], 'Recall': [recall], 'F1': [f1]})
    return pd.concat([results, temp_df], ignore_index=True)


test_scenarios = {
    attack : {'X_test': X_test, 'y_test': y_test}
}

all_results = pd.DataFrame()

for category, scenario in test_scenarios.items():
    X_test_id = scenario['X_test']

    y_test = scenario['y_test']
	
    all_results = evaluate_model(model, X_test, y_test, category, all_results)



csv_file_name = f"{path}/{protocol}_{version}_{attack}.csv"
h5_model_name = f"{path}/{protocol}_{version}_{attack}_model.h5"


all_results.to_csv(csv_file_name, index=False)
model.save(h5_model_name, save_format="tf")

5841/5841 [==============================] - 12s 2ms/step
